# Poseidon: Internet of Things voor waterbesparing

Het Poseidon-project van IBM is een voorbeeld van een IoT toepassing, in dit geval gericht op het besparen van water bij het laten groeien van planten.

Het demonstratie-project bevat typische onderdelen van IoT toepassingen:

* sensoren en actuatoren, in een "things" omgeving
    * meten van vochtigheid van de grond, temperatuur, luchtdruk
    * aansturen van een druppelaar en van een (groei-)lamp
* een computer om deze sensoren en actuatoren uit te lezen en aan te sturen
    * een Raspberry Pi, met een GrovePi interface
* software om deze computer en deze gegevens te koppelen aan het IoT
    * via het MQTT protocol: Mosquitto op RasPi
* server met MQTT - publish/subscribe
* server met web-toepassing

## Links

* [Poseidon-IBM](http://www.ibm.com/developerworks/cloud/library/cl-poseidon1-app/)
* [MQTT](http://mqtt.org)
* [Node-RED](http://nodered.org)
* [Mosquitto](http://mosquitto.org)


## MQTT

MQTT is een publish-subcribe protocol voor het Internet of Things. 

Publish-subscribe houdt in dat de data vanuit de bron (sensoren) gepubliceerd worden naar een centraal punt (de *broker*). Dit centrale punt is bekend onder een naam (URL). Gebruikers van deze data kunnen zich abonneren bij dit centrale punt. Dit betekent dat elke keer als er nieuwe gegevens binnenkomen, deze doorgestuurd worden naar alle subscribers.

* [ ] Op welk lager protocol is MQTT gebaseerd? (Bij mijn weten kan het ook over TLS/SSL - een beveiligde verbinding.)
    * de Python client gebruikt het "kale" MQTT protocol
    * de JavaScript client gebruikt websockets (MQTT bovenop http), voor de verbinding met de broker. Deze client gebruik je bijvoorbeeld voor het verwerken van gepubliceerder sensor-data (subscribe).
* [ ] Is het ook mogelijk om gebruik te maken van HTTP, bijv. in combinatie met websockets?
* [x] MQTT-wiki: https://github.com/mqtt/mqtt.github.io/wiki
* [ ] Wat is de typische latency van een MQTT verbinding (client -> broker -> client)? Kun je dit bijvoorbeeld gebruiken om op afstand op een event te reageren, binnen redelijke tijd?

### Mosquitto

Mosquitto is een implementatie van een MQTT broker, in Python.

Je kunt Mosquitto ook op een Raspberry Pi draaien.

### Paho-mqtt

Het Paho-project biedt Open Source implementaties van een MQTT client, in verschillende talen (o.a. Python, JavaScript). Met een client kun je zowel lokale resultaten publiceren naar een server, als je abonneren (subscribe) op de resultaten van een server (broker).

* [Paho-MQTT](http://eclipse.org/paho/#mqtt)

Opmerkingen:

* het verkeer tussen de client en de server is asynchroon: je weet niet wanneer er een bericht van de server komt. Dit betekent dat er hevig gebruik gemaakt wordt van "callback" functies.

### Clients en servers

Een server moet zichtbaar zijn in het publieke internet.

Een client kan zowel als bron van data fungeren, als als subscriber. Een client hoeft niet zichtbaar te zijn op 

## Node-RED

Node-RED is een manier om node.js servers op te zetten, door op een grafische manier de verschillende onderdelen aan elkaar te koppelen. Zowel node.js als Node-RED zijn redelijk "light weight": je kunt bijvoorbeeld een RasPi als server gebruiken.

### Node-RED op RasPi

Zie: http://nodered.org/docs/hardware/raspberrypi.html voor het installeren van Node-RED op een Raspberry Pi.

> Waarom zou je Node-RED op een Raspberry Pi willen installeren? Je kunt hiermee een lokale server maken. (Meestal is je RasPi niet zichtbaar in het publieke internet. Je kunt deze dan niet gebruiken voor een publieke MQTT server.) Het is ook een goede manier om te oefenene met Node-RED, en met client-server architecturen, bijvoorbeeld in de schoolomgeving.


## Gebruik van BlueMix

BlueMix is de IBM cloud-server-omgeving. Je kunt in de context van dit project deze omgeving gebruiken voor je eigen MQTT server. Je kunt de data die naar deze server opgestuurd worden opslaan in een database.

## Gebruik van Cloud9

Cloud9 (c9.io) is een cloud-gebaseerde ontwikkelomgeving. Als gebruiker heb je de beschikking over een aantal workspaces; elke workspace is een virtuele Linux-machine waarop de nodige software geïnstalleerd is. Je kunt deze omgeving gebruiken voor het ontwikkelen van web-toepassingen. Je kunt deze uittesten via Cloud9. Voor het daadwerkelijk draaien van de toepassing heb je een andere dienst nodig, bijvoorbeeld BlueMix, Amazon AWS of Google.


## Web-toepassing

Wat doe je met de gegevens die je van de verschillende sensoren ontvangt? Je kunt deze bijvoorbeeld zichtbaar maken als een grafiek, in een web-toepassing.

Als je in je toepassing ook gebruik maakt van actuatoren, zoals in het geval van het Poseidon-project, dan kun je deze eventueel ook via een web-toepassing aansturen.

## Sensoren

Hoe vaak moet je meten? Wanneer stuur je deze gegevens naar de server?



## Big Data

De gegevens van de sensoren vormen een mogelijke bron van data voor een "Big Data" activiteit. In het geval van Big Data gaat het om het interpreteren van gegevens.

## Andere vormen van gebruik

### Node-RED als webserver

Je kunt Node-RED gebruiken om een eenvoudige webserver op te zetten. Hiermee kun je een aantal principes van webservers demonstreren. Node-RED is dan een alternatief voor Apache en PHP.

### Messaging via MQTT (je eigen WhatsApp?)

Je kunt denk ik MQTT ook gebruiken voor het distribueren van eenvoudige boodschappen, in een groep. Via de broker kunnen clients zich abonneren op een bepaalde klasse van boodschappen. Via een client kan iedereen ook een boodschap in een bepaalde klasse publiceren.



## Instellingen Poseidonproject

De RasPi sensornode stuur elk uur de waarden van de sensoren op naar de broker. (Dat is wel een zeer lage frequentie - met het oog op een groot aantal gebruikers?)

De broker die ingesteld staat voor de RasPi: `realtime.ngi.ibm.com`, met als (MQTT) subject: `/org/dutchcourage/poseidon/client/sensor`. Deze broker wordt in de Poseidon-bescrhijving ook wel een *micro broker* genoemd.

Deze broker blijft ook gehandhaafd voor de volgende stappen: de Node-RED server fungeert als client van deze broker.

> Wat is de reden om zo'n extra broker ertussen te zetten? Deze combineert, via hetzelfde subject, wel alle sensoren via hetzelfde subject.

### Locatie

* 52.361958, 4.918192

### Startup van MQTT client bij opstarten RasPi

Om ervoor te zorgen dat de gegevens van de sensoren altijd doorgestuurd worden naar de broker, is het handig als de MQTT client automatisch opstart bij de RasPi boot. Hiervoor moeten twee dingen gebeuren:

* het script dat hiervoor zorgt moet in `/etc/init.d` geplaatst worden. Dit script moet aan een aantal voorwaarden voldoen: het moet een *service* zijn, met opdrachten als `start` en `stop`.
    * zie bijv. http://www.tldp.org/HOWTO/HighQuality-Apps-HOWTO/boot.html
* het script moet uitvoerbaar zijn (*executable*)
    * voer hiervoor het commando uit: `chmod 755 myserver.sh`
* het script moet aangemeld worden als bootup-service, via `update-rc.d`
    * `sudo update-rc.d myserver.sh default`
    * (wat doet dit commando? wordt alleen de naam van de service gebruikt, of is dit een lokale filenaam? Onder Unix zou je toch dit laatste verwachten...)
    

## OS X

* Installatie van mosquitto op OS X: `/usr/local/Cellar/mosquitto/1.3.1_1`


## Google Cloud

* Project ID: dynamic-shift-95520 

## IBM Cloud

* ik krijg een foutmelding, bij de Deploy van de Node-RED flow.
    * "TypeError: Cannot call method 'on' of null"
    * https://developer.ibm.com/answers/questions/18881/iot-node-red-debug-window-not-showing-data-from-simulated-device.html
* NB: er zijn twee Cloudant nodes; je moet de node hebben die de message opslaat in de DB. (Dat merk je snel genoeg: de andere node is voor het ophalen van gegevens uit de DB. Dan heb je niet te maken met een "insert" maar met een "search".) 

## Cloud9

Cloud9 is een ontwikkelomgeving "in the cloud". Je hebt per project de beschikking over een eigen omgeving (virtuele machine) waarin je kunt ontwikkelen (met een editor), en waarin je web-servers kunt uitvoeren.

Je hebt enkele beperkingen:

* je kunt de UR, het IP-adres, en het port-nummer niet aanpassen: dit moet je overnemen uit de gegeven variabelen.
    * dit betekent dat je wel web-diensten kunt uitproberen, maar geen willekeurige internet-diensten. Mosquitto kun je er niet draaien (tenzij over websockets)
* voor Node-RED heb je een aparte port voor de flow-editor. In Cloud9 moet je daarvoor de standaard-port gebruiken: in dit geval deel je dezelfde port voor de flow-editor en voor de eigenlijke dienst.
* voor het gebruik van MQTT moet je ook over andere PORTs kunnen beschikken?

Aanpassingen in `settings.js`:

* `uiPort: process.env.PORT, // was: 1880`
* `uiHost: process.env.IP,   // was: "127.0.0.1"`

Commando: `node-red --settings settings.js`


## Oefenen met Node-RED

Met Node-RED kun je je eigen (Node.js) webserver maken: je kunt voor elk verzoek (GET, met een bepaalde URL) een "flow" definiëren.

In een omgeving waarin je een volledige server (virtuele machine) tot je beschikking hebt, gebruik je een aparte port voor de flow-editor, naast de normale port voor de eigenlijke webserver. Maar in het geval van Cloud9 heb je die luxe niet: je gebruikt dan dezelfde port voor de flow-editor als voor de eigenlijke webserver.


## Gebruik van Raspberry Pi

Je kunt de Raspberry Pi ook gebruiken voor deze verschillende elementen:

* je kunt Mosquitto als MQTT broker draaien op een RasPi.
    * dit zou je bijvoorbeeld in de klas kunnen doen, om de verschillende Raspberry Pi's (en Arduino's?) te laten communiceren.
    * bij voorkeur zou je een Raspberry Pi hiervoor als server in het publieke internet verbinden, maar meestal heb je daarvoor geen mogelijkheden. Soms kun je dit via een NAT-instelling op een router doen. Je moet er dan rekening mee houden dat je voor MQTT een andere poort nodig hebt.
    * wij zouden deze oplossing voor ICT in de Wolken kunnen gebruiken: we hebben wel een router die we hiermee kunnen aanpassen.
* je kunt Node-RED op een Raspberry Pi draaien. Hiermee kun je in de klas demonstreren de werking van een webserver demonstreren. (Dit kun je ook vergelijken met het gebruik van een webserver voor het bedienen van een apparaat.)
* je kunt node.js op een Raspberry Pi draaien - als voorbeeld van een webserver.
    * hier geldt ook weer hetzelfde voorbehoud als bij MQTT: je hebt het liefst een publieke verbinding.
* kun je in de klas ook een DNS dienst opzetten, om te voorkomen dat iedereen steeds IP-adressen moet opgeven?

    

## Acties

### Raspberry Pi: nieuwe SD kaart

Als je begint met een nieuwe geheugenkaart (SD of micro SD), is het handig om de systeeminstellingen aan te passen, en de software bij te werken naar de nieuwste versie. Zorg ervoor dat je RasPi is aangesloten op het internet.

1. het programma voor het instellen van de Raspberry Pi start automatisch (grijze achtergrond). Stel hiermee in elk geval de tijdzone in. Je kunt eventueel ook het wachtwoord aanpassen.
    * de default instellingen zijn username: `pi` en password: `raspberry`.
2. de volgende stap is het bijwerken van de software naar de nieuwste versie. wanneer je de prompt van de commandline krijgt, voer dan de volgende opdrachten uit:
    * `sudo apt-get update` - haalt een beschrijving op van de nieuwste versies
    * `sudo apt-get upgrade` - werkt de software bij volgens de opgehaalde beschrijvingen
3. herstart het systeem na dit bijwerken, met de opdracht:
    * `sudo shutdown -r now`

Het is verstandig om de software regelmatig bij te werken, met de combinatie zoals onder punt 2 beschreven.
    


### Installeren Arduino software

De Arduino software gebruik je om een Arduino op de Raspberry Pi aan te sluiten, en te programmeren. Ook voor de GrovePi is deze software handig: eigenlijk is dit een Arduino-systeem dat je als "shield" op een Raspberry Pi kunt plaatsen.

* `sudo apt-get install arduino`

#### Controleren Arduino software

1. start de desktop op:
    * `startx`
2. start met het menu links de Arduino IDE op (bijvoorbeeld via Elektronica).

### Installeren GrovePi

Voor het installeren van de GrovePi software volg je de handleiding zoals beschreven op de website: XXX. Zorg ervoor dat je systeem op het internet is aangesloten, en dat het GrovePi bord nog *niet in aangesloten* op de Raspberry Pi.Het installatiescript vraagt een aantal malen om bevestiging (RETURN of y RETURN). Het script installeert de software, past het systeem aan, en start het systeem opnieuw op.

#### Controleren GrovePi

Voor het controleren van de GrovePi software en hardware moeten we de GrovePi hardware aansluiten. *Verander alleen iets aan de harware van de Raspberry Pi als deze helemaal uit is: voeding (micro USB) niet aangesloten.* 

1. stop de Raspberry Pi: `sudo shutdown -h now`
2. als het systeem uit is (monitor in sleep, 10 sec. geen activiteit op RasPi), verwijder dan de stroomvoorziening (micro USB).
3. plaats nu voorzichtig het GrovePi shield op de Raspberry Pi. Zorg ervoor dat de GrovePi-connector recht boven de pinnen van de Raspberry Pi geplaatst is.
4. sluit nu de stroomvoorziening weer aan: het systeem start opnieuw op.
5. start de desktop op (`startx`)
6. start een terminal, en voor daarin de opdracht uit: 

In de handleiding op de GrovePi website staat het resultaat beschreven dat je nu zou moeten zien.


### Desktop en terminal

Als je de Raspberry Pi opstart, kom je in de command-line mode: de "shell". Vaak is het handig om vanuit de desktop te werken. Je start deze op met de opdracht: 

* `startx`

In de desktop kun je één of meerdere terminals opstarten, om shell-opdrachten uit te kunnen voeren.


## Cheatsheet

Enkele van de belangrijkste opdrachten in de shell, voor dit project:

| opdracht | betekenis |
| :---     |           |
| `sudo shutdown -r now` | stop en herstart het systeem (reboot) |
| `sudo showdown -h now` | stop het systeem (halt) |
| `sudo apt-get update`  | haal de beschrijving van de nieuwste software-versies op |
| `sudo apt-get upgrade` | werk de software bij volgens deze beschrijving |
| `sudo apt-get install XXX` | installeer programma `XXX` (bijvoorbeeld `arduino`) |
| `sudo reboot` | stop en herstart het systeem |
| `ls`                   | beknopte lijst van bestanden in huidige directory |
| `ls -l`                | uitgebreide lijst van bestanden in huidige directory |

